## Preprocessing

In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Trial 1 'Binning and encoding for app type  < 100 and for classification  < 100 ,number of Layers 128-64-32 ', epochs=100 ,fct activation relu and segment for output

# Drop non-beneficial columns
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

# Binning and encoding for APPLICATION_TYPE
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(app_type_counts[app_type_counts < 300].index)

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Binning and encoding for CLASSIFICATION
class_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(class_counts[class_counts < 80].index)

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Convert categorical data to numeric with pd.get_dummies
application_df_encoded = pd.get_dummies(application_df, dtype=float)

# Split the data
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model
num_input_features = len(X_train_scaled[0])
hid_nodes_layer1 = 128
hid_nodes_layer2 = 64
hid_nodes_layer3 = 32

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=hid_nodes_layer1, input_dim=num_input_features, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hid_nodes_layer2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hid_nodes_layer3, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model with different epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, verbose=2)


Epoch 1/100
804/804 - 2s - loss: 0.5688 - accuracy: 0.7222 - 2s/epoch - 3ms/step
Epoch 2/100
804/804 - 1s - loss: 0.5527 - accuracy: 0.7300 - 1s/epoch - 2ms/step
Epoch 3/100
804/804 - 1s - loss: 0.5496 - accuracy: 0.7330 - 1s/epoch - 2ms/step
Epoch 4/100
804/804 - 1s - loss: 0.5486 - accuracy: 0.7320 - 1s/epoch - 2ms/step
Epoch 5/100
804/804 - 2s - loss: 0.5465 - accuracy: 0.7335 - 2s/epoch - 2ms/step
Epoch 6/100
804/804 - 2s - loss: 0.5454 - accuracy: 0.7334 - 2s/epoch - 2ms/step
Epoch 7/100
804/804 - 2s - loss: 0.5446 - accuracy: 0.7354 - 2s/epoch - 2ms/step
Epoch 8/100
804/804 - 1s - loss: 0.5440 - accuracy: 0.7364 - 1s/epoch - 2ms/step
Epoch 9/100
804/804 - 1s - loss: 0.5429 - accuracy: 0.7353 - 1s/epoch - 2ms/step
Epoch 10/100
804/804 - 1s - loss: 0.5427 - accuracy: 0.7368 - 1s/epoch - 2ms/step
Epoch 11/100
804/804 - 1s - loss: 0.5418 - accuracy: 0.7367 - 1s/epoch - 2ms/step
Epoch 12/100
804/804 - 1s - loss: 0.5418 - accuracy: 0.7380 - 1s/epoch - 2ms/step
Epoch 13/100
804/804 - 1s

In [21]:

# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5678 - accuracy: 0.7270 - 469ms/epoch - 2ms/step
Loss: 0.5678246021270752, Accuracy: 0.7269970774650574


In [22]:
# Export our model to HDF5 file
from google.colab import files

nn.save('AlphabetSoupCharity_optimisation1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
# Trial 2 'modification Binning and encoding for app type  < 100 and for classification  < 100 ,modify number of Layers 80-30-20 ', epochs=150 ,fct activation relu and segment for output
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()
# Drop non-beneficial columns
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

# Binning and encoding for APPLICATION_TYPE
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(app_type_counts[app_type_counts < 100].index)

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Binning and encoding for CLASSIFICATION
class_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(class_counts[class_counts < 100].index)

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Convert categorical data to numeric with pd.get_dummies
application_df_encoded = pd.get_dummies(application_df, dtype=float)

# Split the data
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model
num_input_features = len(X_train_scaled[0])
hid_nodes_layer1 = 80
hid_nodes_layer2 = 30
hid_nodes_layer3 = 20

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=hid_nodes_layer1, input_dim=num_input_features, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hid_nodes_layer2, activation="relu"))
nn.add(tf.keras.layers.Dense(units=hid_nodes_layer3, activation="relu"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model with different epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=150, verbose=2)


Epoch 1/150
804/804 - 3s - loss: 0.5727 - accuracy: 0.7172 - 3s/epoch - 3ms/step
Epoch 2/150
804/804 - 2s - loss: 0.5520 - accuracy: 0.7319 - 2s/epoch - 2ms/step
Epoch 3/150
804/804 - 1s - loss: 0.5494 - accuracy: 0.7306 - 1s/epoch - 2ms/step
Epoch 4/150
804/804 - 1s - loss: 0.5484 - accuracy: 0.7318 - 1s/epoch - 2ms/step
Epoch 5/150
804/804 - 1s - loss: 0.5468 - accuracy: 0.7318 - 1s/epoch - 2ms/step
Epoch 6/150
804/804 - 1s - loss: 0.5464 - accuracy: 0.7337 - 1s/epoch - 2ms/step
Epoch 7/150
804/804 - 1s - loss: 0.5445 - accuracy: 0.7355 - 1s/epoch - 2ms/step
Epoch 8/150
804/804 - 1s - loss: 0.5445 - accuracy: 0.7343 - 1s/epoch - 2ms/step
Epoch 9/150
804/804 - 1s - loss: 0.5435 - accuracy: 0.7358 - 1s/epoch - 2ms/step
Epoch 10/150
804/804 - 1s - loss: 0.5427 - accuracy: 0.7360 - 1s/epoch - 2ms/step
Epoch 11/150
804/804 - 2s - loss: 0.5420 - accuracy: 0.7371 - 2s/epoch - 2ms/step
Epoch 12/150
804/804 - 2s - loss: 0.5420 - accuracy: 0.7367 - 2s/epoch - 2ms/step
Epoch 13/150
804/804 - 1s

In [7]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5610 - accuracy: 0.7248 - 497ms/epoch - 2ms/step
Loss: 0.5609777569770813, Accuracy: 0.724781334400177


In [8]:
# Export our model to HDF5 file
from google.colab import files

nn.save('AlphabetSoupCharity_optimisation2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Trial 3 'Binning and encoding for app type  < 100 and for classification  < 100 ,modify number of Layers 80-30-20 ', modify epochs=100 ,fct activation tanh and segment for output
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()
# Drop non-beneficial columns
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

# Binning and encoding for APPLICATION_TYPE
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(app_type_counts[app_type_counts < 100].index)

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Binning and encoding for CLASSIFICATION
class_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(class_counts[class_counts < 100].index)

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

# Convert categorical data to numeric with pd.get_dummies
application_df_encoded = pd.get_dummies(application_df, dtype=float)

# Split the data
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Define the model
num_input_features = len(X_train_scaled[0])
hid_nodes_layer1 = 80
hid_nodes_layer2 = 30
hid_nodes_layer3 = 20

nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=hid_nodes_layer1, input_dim=num_input_features, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=hid_nodes_layer2, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=hid_nodes_layer3, activation="tanh"))
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model with different epochs
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, verbose=2)


Epoch 1/100
804/804 - 2s - loss: 0.5671 - accuracy: 0.7239 - 2s/epoch - 3ms/step
Epoch 2/100
804/804 - 1s - loss: 0.5540 - accuracy: 0.7301 - 1s/epoch - 2ms/step
Epoch 3/100
804/804 - 1s - loss: 0.5498 - accuracy: 0.7308 - 1s/epoch - 2ms/step
Epoch 4/100
804/804 - 1s - loss: 0.5480 - accuracy: 0.7327 - 1s/epoch - 2ms/step
Epoch 5/100
804/804 - 1s - loss: 0.5463 - accuracy: 0.7336 - 1s/epoch - 2ms/step
Epoch 6/100
804/804 - 2s - loss: 0.5447 - accuracy: 0.7348 - 2s/epoch - 2ms/step
Epoch 7/100
804/804 - 2s - loss: 0.5446 - accuracy: 0.7334 - 2s/epoch - 3ms/step
Epoch 8/100
804/804 - 1s - loss: 0.5433 - accuracy: 0.7346 - 1s/epoch - 2ms/step
Epoch 9/100
804/804 - 1s - loss: 0.5427 - accuracy: 0.7341 - 1s/epoch - 2ms/step
Epoch 10/100
804/804 - 1s - loss: 0.5419 - accuracy: 0.7369 - 1s/epoch - 2ms/step
Epoch 11/100
804/804 - 1s - loss: 0.5415 - accuracy: 0.7360 - 1s/epoch - 2ms/step
Epoch 12/100
804/804 - 1s - loss: 0.5404 - accuracy: 0.7373 - 1s/epoch - 2ms/step
Epoch 13/100
804/804 - 1s

In [11]:
# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5545 - accuracy: 0.7263 - 773ms/epoch - 3ms/step
Loss: 0.554489254951477, Accuracy: 0.7262973785400391


In [12]:
# Export our model to HDF5 file
from google.colab import files

nn.save('AlphabetSoupCharity_optimisation3.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
